# Load Pretrain Model

In [29]:
import os
import torch 
import torch.nn as nn
import torchvision
import sys
import subprocess

def replace_head(model, num_classes):
    if num_classes == 0:
        model.fc = nn.Identity()
    else: 
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model 

def get_model_weights(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=True)
    return checkpoint['model']

# Use TorchVision Models here 
def load_pretrained_model(model_name, num_classes, device, offical_pretrain=False, pretrain_path=None):
    # only load image-net models
    # change the head to num_classes
    # if num_classes == 0 then remove the head 
    if not offical_pretrain:
        model = torchvision.models.get_model(model_name, pretrained=False)
        model_weights = get_model_weights(pretrain_path)
        model.load_state_dict(model_weights)
    else: 
        model = torchvision.models.get_model(model_name, pretrained=True)
    model = replace_head(model, num_classes)
    model = model.to(device)

    for param in model.parameters():
        param.requires_grad = True
    return model 

def load_model(model_name, num_classes, device):
    model = torchvision.models.get_model(model_name, pretrained=False)
    model = replace_head(model, num_classes)
    model = model.to(device)
    return model

## CUB200 Dataset

In [2]:
import os
import pandas as pd
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset


class Cub2011(Dataset):
    base_folder = 'CUB_200_2011/images'
    url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'
    filename = 'CUB_200_2011.tgz'
    tgz_md5 = '97eceeb196236b17998738112f37df78'

    def __init__(self, root, train=True, transform=None, loader=default_loader, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.loader = default_loader
        self.train = train

        if download:
            self._download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

    def _load_metadata(self):
        images = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'images.txt'), sep=' ',
                             names=['img_id', 'filepath'])
        image_class_labels = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'image_class_labels.txt'),
                                         sep=' ', names=['img_id', 'target'])
        train_test_split = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'train_test_split.txt'),
                                       sep=' ', names=['img_id', 'is_training_img'])

        data = images.merge(image_class_labels, on='img_id')
        self.data = data.merge(train_test_split, on='img_id')

        if self.train:
            self.data = self.data[self.data.is_training_img == 1]
        else:
            self.data = self.data[self.data.is_training_img == 0]

    def _check_integrity(self):
        try:
            self._load_metadata()
        except Exception:
            return False

        for index, row in self.data.iterrows():
            filepath = os.path.join(self.root, self.base_folder, row.filepath)
            if not os.path.isfile(filepath):
                print(filepath)
                return False
        return True

    def _download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        download_url(self.url, self.root, self.filename, self.tgz_md5)

        with tarfile.open(os.path.join(self.root, self.filename), "r:gz") as tar:
            tar.extractall(path=self.root)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        path = os.path.join(self.root, self.base_folder, sample.filepath)
        target = sample.target - 1  # Targets start at 1 by default, so shift to 0
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

Some helper functions

In [3]:
def seed_everything(seed=3407):
    """
    Sets the seed for Python's random module, NumPy, and PyTorch to ensure reproducibility.

    Parameters:
    - seed: An integer used as the seed for random number generators. Default is 3407.

    Raises:
    - ImportError: If torch is not installed.
    """
    import random
    import os 
    import numpy as np
    try:
        random.seed(seed)
        os.environ["PYTHONHASHSEED"] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = True
        else:
            print("CUDA is not available. Seeding only for CPU.")
    except ImportError:
        print("torch is not installed. Seeding only for Python's random module and NumPy.")



In [4]:
def install_package(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} has been installed")

In [5]:
!ls weights/

In [37]:
DATA_NAME = 'Flowers102'
EPOCHS = 100
BATCH_SIZE = 64 
LR_BEGIN = 0.01
PRETRAIN = False
MODEL_NAME = 'resnet50'



In [38]:
from torchvision import transforms
from torch.utils.data import Dataset, ConcatDataset

re_size = 512
crop_size = 448
train_transform = transforms.Compose(
    [
        transforms.Resize((re_size, re_size)),
        transforms.RandomCrop(crop_size, padding=8),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)
test_transform = transforms.Compose(
    [
        transforms.Resize((re_size, re_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)


def load_dataset(dataset_name, train_transform, test_transform):
    if dataset_name == 'CUB200':
        print("Loading CUB200")
        train_set = Cub2011(root='./data', train=True, transform=train_transform)
        test_set = Cub2011(root='./data', train=False, transform=test_transform)
    print("Loading", dataset_name)
    train_set = getattr(torchvision.datasets, dataset_name)(root='./data', split='train', download=True, transform=train_transform)
    if dataset_name == 'Flowers102':
        train_set_2 = getattr(torchvision.datasets, dataset_name)(root='./data', split='val', download=True, transform=train_transform)
        concat_dataset = ConcatDataset([train_set, train_set_2])
        train_set = concat_dataset
    test_set = getattr(torchvision.datasets, dataset_name)(root='./data', split='test', download=True, transform=test_transform)
    print("train set size:", len(train_set))
    print("test set size:", len(test_set))
    return train_set, test_set

def get_dataloaders(train_set, test_set, batch_size, num_workers):
    train_sampler = torch.utils.data.RandomSampler(train_set)
    test_sampler = torch.utils.data.SequentialSampler(test_set)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers, sampler=test_sampler)
    return train_loader, val_loader



In [39]:
def get_num_classes(dataset_name):
    if dataset_name == 'Flowers102':
        return 102
    elif dataset_name == 'Food101':
        return 101
    elif dataset_name == 'CUB200':
        return 200
    else: 
        raise Exception(f"Dataset {dataset_name} not supported")

In [40]:
train_set, test_set = load_dataset('Flowers102', train_transform, test_transform)
train_loader, val_loader = get_dataloaders(train_set, test_set, BATCH_SIZE, 6)

Loading Flowers102
train set size: 2040
test set size: 6149


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
number_of_classes = get_num_classes(DATA_NAME)
print(f"Number of classes: {number_of_classes}")
if PRETRAIN:
    model = load_pretrained_model(
        model_name=MODEL_NAME, offical_pretrain=True, num_classes=number_of_classes, device=device, pretrain_path=None
    )
else: 
    model = load_model(model_name=MODEL_NAME, num_classes=number_of_classes, device=device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.SGD(
    model.parameters(), lr=LR_BEGIN, momentum=0.9, weight_decay=5e-4
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=128)



Number of classes: 102


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
from tqdm import tqdm

model.cuda()
min_train_loss = float('inf')
max_eval_acc = 0

scaler = torch.cuda.amp.GradScaler()

for epoch in range(EPOCHS):
    print('\n===== Epoch: {} ====='.format(epoch))
    model.train()  # set model to train mode, enable Batch Normalization and Dropout
    lr_now = optimizer.param_groups[0]['lr']
    train_loss = train_correct = train_total = idx = 0

    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, ncols=80)):
        idx = batch_idx

        if inputs.shape[0] < BATCH_SIZE:
            continue

        optimizer.zero_grad()  # Sets the gradients to zero
        inputs, targets = inputs.cuda(), targets.cuda()
        #print(inputs.shape, targets.shape)

        with torch.cuda.amp.autocast():
            x = model(inputs)
            loss = criterion(x, targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        _, predicted = torch.max(x.data, 1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets.data).cpu().sum()
        train_loss += loss.item()

    scheduler.step()

    train_acc = 100.0 * float(train_correct) / train_total
    train_loss = train_loss / (idx + 1)
    print(
        'Train | lr: {:.4f} | Loss: {:.4f} | Acc: {:.3f}% ({}/{})'.format(
            lr_now, train_loss, train_acc, train_correct, train_total
        )
    )

    ##### Evaluating model with test data every epoch
    with torch.no_grad():
        model.eval()  # set model to eval mode, disable Batch Normalization and Dropout
        eval_loader = val_loader
        eval_correct = eval_total = 0
        for _, (inputs, targets) in enumerate(tqdm(eval_loader, ncols=80)):
            inputs, targets = inputs.cuda(), targets.cuda()
            x = model(inputs)
            _, predicted = torch.max(x.data, 1)
            eval_total += targets.size(0)
            eval_correct += predicted.eq(targets.data).cpu().sum()
        eval_acc = 100.0 * float(eval_correct) / eval_total
        print(
            '{} | Acc: {:.3f}% ({}/{})'.format(
                'Val', eval_acc, eval_correct, eval_total
            )
        )





===== Epoch: 0 =====


  0%|                                                    | 0/32 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 32/32 [00:23<00:00,  1.37it/s]


Train | lr: 0.0250 | Loss: 5.8491 | Acc: 0.806% (16/1984)


100%|███████████████████████████████████████████| 97/97 [00:48<00:00,  2.01it/s]


...

100%|███████████████████████████████████████████| 32/32 [00:23<00:00,  1.35it/s]


Train | lr: 0.0032 | Loss: 0.8571 | Acc: 99.950% (1983/1984)


100%|███████████████████████████████████████████| 97/97 [00:48<00:00,  2.00it/s]


Val | Acc: 57.538% (3538/6149)

===== Epoch: 99 =====


  0%|                                                    | 0/32 [00:00<?, ?it/s]